# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [78]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [79]:
weather_data = pd.read_csv("WeatherPy.csv", encoding="utf-8")
weather_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Cravo Norte,85,CO,1584491531,34,6.30,-70.20,89.06,7.14
1,Nanortalik,17,GL,1584491531,55,60.14,-45.24,19.78,10.92
2,Mount Isa,0,AU,1584491531,35,-20.73,139.50,84.20,12.75
3,George Town,20,MY,1584491237,83,5.41,100.34,78.80,5.82
4,Butaritari,0,KI,1584491532,75,3.07,172.79,83.35,12.68




### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [80]:
weather_data.dtypes

locations = weather_data[["Lat", "Lng"]].astype(float)

humidity = weather_data["Humidity"].astype(float)




In [81]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [82]:
weather_data.dtypes

City           object
Cloudiness      int64
Country        object
Date            int64
Humidity        int64
Lat           float64
Lng           float64
Max Temp      float64
Wind Speed    float64
dtype: object

In [83]:
weather_data=weather_data.loc[(weather_data["Max Temp"]<80) & (weather_data["Max Temp"]>70)
                             &(weather_data["Cloudiness"].astype(float)==0)&
                             (weather_data["Wind Speed"]<10)]

hotel_df=weather_data.dropna()
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
23,Tālcher,0,IN,1584491535,49,20.95,85.22,74.39,2.39
70,Samdari,0,IN,1584491543,26,25.82,72.58,70.27,9.04
223,Kiama,0,AU,1584491570,64,-34.68,150.87,75.99,8.05
248,Airai,0,TL,1584491209,67,-8.93,125.41,72.90,4.52
304,Puri,0,IN,1584491585,83,19.80,85.85,77.00,3.36
337,Grenfell,0,AU,1584491593,44,-33.90,148.17,71.98,5.57
344,Ngunguru,0,NZ,1584491594,48,-35.62,174.50,71.01,4.00
351,Bilma,0,NE,1584491596,9,18.69,12.92,73.40,9.51
402,Tapejara,0,BR,1584491606,71,-28.07,-52.01,71.92,3.24
424,Antalaha,0,MG,1584491609,84,-14.90,50.28,78.37,5.10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [84]:
hotel_df["Hotel Name"] = "" 

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

for index,row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    #print(params["location"])

    # make request and print url
    hotel_response = requests.get(base_url, params=params)
    hotel_response = hotel_response.json()

    from pprint import pprint
    pprint(hotel_response)
        
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_response["results"][0]["name"]
    except: 
        hotel_df.loc[index, "Hotel Name"]="City Name is Wrong"
        



{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': 20.934451, 'lng': 85.225759},
                           'viewport': {'northeast': {'lat': 20.93564602989272,
                                                      'lng': 85.22707762989272},
                                        'southwest': {'lat': 20.93294637010727,
                                                      'lng': 85.22437797010728}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
              'id': 'd4d89d4e944acff81c0fbf4f97faabfac2b9aae2',
              'name': 'Hotel Surya Garden',
              'opening_hours': {'open_now': True},
              'photos': [{'height': 638,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/101807989280575548634">Hotel '
                                                'Surya Garden</a>'],
                          'photo_refe

{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': 25.816373,
                                        'lng': 72.58307350000001},
                           'viewport': {'northeast': {'lat': 25.81753552989272,
                                                      'lng': 72.58450597989273},
                                        'southwest': {'lat': 25.81483587010728,
                                                      'lng': 72.58180632010729}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
              'id': '0402b0034b14b43eac91d0963004725aec7aad6a',
              'name': 'Sonji Ki Hotel',
              'place_id': 'ChIJN1b5S_bJQzkRtayRhAker0E',
              'plus_code': {'compound_code': 'RH8M+G6 Samdari, Rajasthan, '
                                             'India',
                            'global_code': '7JQJRH8M+G6'},
              'rating': 0,
              'reference': 'ChIJN1b5S_bJQzkRtayRhAker0E',


{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [],
 'next_page_token': 'CrQCIwEAAPoAy6ub1perTKtzwlasXWYrHCPt9_qPEGWQzgojvHHdAf1kwI07xxH2XgTJzKZT_5R9bQ8V9MNkSIloK_vFjEUQHxzyXm-0XpvatfFoznfb1jJqqUEe5-DSehVa-PjqdDJxOANfDmkt0G1UzYR8qgUjmKZ83B3X50AFYanNX28eJz3PeefP3L9itNeoBqX7Mo5MgnrUtFDS3-yirnkFgFNaL_BgQRsIFsIJyET9GW4mVmCYSvYnzZlDrGzrkDEhH_rYAfg9HWIAUWSVsertDmVIOlOPPS9TpJxTlr_GAciRoN7qcGozqIcTtpGiRnQdLn8sO6Zh4WPbP1E2F98nJqal5Vm7RiurysVUNCTwmSBkyERhvinOjebvisWNYKkn0z3qxFsLZwgOgss_-fv20ZISEJuqgoI4-B6vA5YtVTEXx28aFBWcp-Lv98S0t3Dqnqe1PQIvtBAN',
 'results': [{'geometry': {'location': {'lat': 19.801556, 'lng': 85.8413047},
                           'viewport': {'northeast': {'lat': 19.80293637989272,
                                                      'lng': 85.84276577989272},
                                        'southwest': {'lat': 19.80023672010728,
                                                      'lng': 85.84006612010727}}},
             

{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': -33.8980556, 'lng': 148.1616667},
                           'viewport': {'northeast': {'lat': -33.89673567010728,
                                                      'lng': 148.1632868298927},
                                        'southwest': {'lat': -33.89943532989273,
                                                      'lng': 148.1605871701072}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
              'id': 'b85c97a268dcc884f45109939e200f65f671900f',
              'name': 'Grenfell Hall',
              'photos': [{'height': 2048,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/101084542429876508267">Elizabeth '
                                                'S</a>'],
                          'photo_reference': 'CmRaAAAAH0f2GTe3BuBLmL3Y6U_TI7iBaloV1rgcDBoRbIV

{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': -35.6099474, 'lng': 174.5245359},
                           'viewport': {'northeast': {'lat': -35.60862607010726,
                                                      'lng': 174.5259724298927},
                                        'southwest': {'lat': -35.61132572989271,
                                                      'lng': 174.5232727701073}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
              'id': '5cd4496dffa781822a256b00f91738b32136de5d',
              'name': 'Quality Hotel Oceans Tutukaka',
              'opening_hours': {'open_now': True},
              'photos': [{'height': 1688,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/106233846196630525364">Oceans '
                                                'Resort Hotel</a>'],
                  

{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': -14.9061237, 'lng': 50.2785486},
                           'viewport': {'northeast': {'lat': -14.90481452010728,
                                                      'lng': 50.27988962989272},
                                        'southwest': {'lat': -14.90751417989272,
                                                      'lng': 50.27718997010728}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
              'id': 'c869993e16cce59227e9b9e7aa89b468e03c7156',
              'name': 'Villa-Malaza',
              'photos': [{'height': 1536,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/102604115630443166232">A '
                                                'Google User</a>'],
                          'photo_reference': 'CmRaAAAAoywniprBzCq7gJT03OEmP2lLs4X7Pjy-6vX6ojR

{'html_attributions': [],
 'next_page_token': 'CrQCIwEAAJ_BdvTbfGIejIynGQcRuMyIRn6GJspyjk8BIVGG73vEom6Xh8mpgSFwRmzWC-TKoecgpC1yKt8BRQYgZzh4xj-KmTswHFnvaO-dMUblxP5a7yZThvAvFuXoNAujiwJLbt7Al-15sed-2TISLWU6v8mxB0DipFoTsbKLYc7PwLD_RjQ4CuKqdT9mWT2NiTb9s_wp6Efn1te-gVt2nOtPcrpF4jfMLDewgUkIYbyvBvsATXSEjZ52sdVFoFPBRwJST5RriwW3zk4bmVgpx7AVNyGVDbFyjmARLvzAoYh1-3IUPsdwldLReW2r2uPZZJN2pGXQuMPeWC6ghU3oWvJMop60QDlQhxRd6MPngffhf0dyvfW8HTp4eR-g3ZHh5JQwOs8nQmtCkkshYiphLJBJGXISEDlMkJypo8-daPyyn0q9E70aFGb4hs6LZbdQfcVO6cBn7pfbqmhV',
 'results': [{'geometry': {'location': {'lat': 21.6381752, 'lng': 69.6051877},
                           'viewport': {'northeast': {'lat': 21.63951597989272,
                                                      'lng': 69.60657617989273},
                                        'southwest': {'lat': 21.63681632010728,
                                                      'lng': 69.60387652010728}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodgin

{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': 12.4870317, 'lng': -16.5444956},
                           'viewport': {'northeast': {'lat': 12.48840017989272,
                                                      'lng': -16.54312672010727},
                                        'southwest': {'lat': 12.48570052010728,
                                                      'lng': -16.54582637989272}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
              'id': '060ad91f83de3800cd2cfe6e6aa8bb1f35784783',
              'name': 'Hotel Carabane',
              'photos': [{'height': 4160,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/113439272708993034228">Modou '
                                                'Samb Modou Samb</a>'],
                          'photo_reference': 'CmRaAAAAqpnMZmFo030NEqnCe_VsElQVcE2ox

In [85]:
hotel_df.dropna()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
23,Tālcher,0,IN,1584491535,49,20.95,85.22,74.39,2.39,Hotel Surya Garden
70,Samdari,0,IN,1584491543,26,25.82,72.58,70.27,9.04,Sonji Ki Hotel
223,Kiama,0,AU,1584491570,64,-34.68,150.87,75.99,8.05,The Grand Hotel
248,Airai,0,TL,1584491209,67,-8.93,125.41,72.90,4.52,City Name is Wrong
304,Puri,0,IN,1584491585,83,19.80,85.85,77.00,3.36,Hotel Holiday Resort
337,Grenfell,0,AU,1584491593,44,-33.90,148.17,71.98,5.57,Grenfell Hall
344,Ngunguru,0,NZ,1584491594,48,-35.62,174.50,71.01,4.00,Quality Hotel Oceans Tutukaka
351,Bilma,0,NE,1584491596,9,18.69,12.92,73.40,9.51,City Name is Wrong
402,Tapejara,0,BR,1584491606,71,-28.07,-52.01,71.92,3.24,Gaúcho Hotel
424,Antalaha,0,MG,1584491609,84,-14.90,50.28,78.37,5.10,Villa-Malaza


In [86]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [76]:
# Add marker layer ontop of heat map

hotel_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
#markers = gmaps.marker_layer(locations,hotel_info)

fig.add_layer(hotel_layer)
fig


# Display Map

Figure(layout=FigureLayout(height='420px'))